In [1]:
%load_ext dotenv
%dotenv

In [2]:
from sklearn.pipeline import Pipeline 
from pipeline import group, photos, user, posts, utils

In [3]:
clearing = Pipeline([
    ('user_id', user.GetUserIdPipelineV2()),
    ('user_posts', posts.GetPostsFromUserPipelineV2(max_repeat_count=0, only_user_text=True)),
])
tf = clearing.transform(["https://vk.com/nastia_z"]) 

In [4]:
text_worker = Pipeline([
    ('user_id', utils.ClearTextPipeline()),
])

In [5]:
tff =  text_worker.transform(tf)

In [6]:
tff

[{'user_id': 74140961,
  'content': {'wall': ['дипломированный экономист дипломированный экономист дипломированный экономист дипломированный экономист дипломированный экономист дипломированный экономист дипломированный экономист дипломированный экономист дипломированный экономист но экономить до сих пор не умеюдипломированный экономист но экономить до сих пор не умею',
    'хочется простого женского: на море хочется простого женского на море хочется простого женского: на море хочется простого женского: на море хочется простого женского: на море хочется простого женского: на море хочется простого женского: на море хочется простого женского: на море хочется простого женского: на море хочется простого женского: на море ',
    '',
    'сияйсияйсияйсияйсияйсияйсияйсияйсияйсияй',
    'июльиюльиюльиюльиюльиюльиюльиюльиюльиюль',
    'за карантин набралось столько событий, которые надо отметить с родными людьми, что теперь не знаю, что и делать отмечать все в один день здоровья не хватит, а есл

In [82]:
from pipeline.statistics import autoaression 

In [94]:
import re
def deEmojify(text):
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    return regrex_pattern.sub(r'',text)

def mister_cleaner(text):
    text = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', text)
    text = text.lower()
    text = re.sub("#(\w+)", " ", text)
    text = re.sub("([\(\[]).*?([\)\]])", "", text)
    text = text.replace("]", "")
    text = text.replace("[", "")
    text = text.replace("(", "")
    text = text.replace(")", "")
    text = text.split('\n')
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in text]
    
    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]
    text = [re.sub("\'", "", sent) for sent in data]
    text = ' '.join(text)
    return text

In [95]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
class AutoreggressionPipeline(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for user in X:
            group_info = []
            for id in user.keys():
                group_result = []
                for group in user[id]:
                    for group_id in group:# 2-ой подуровень групп
                        group_anaysis = {
                                'spontaneous_aggression':[],
                                'depression':[],
                                'balance':[],
                                'emotional_lability':[]
                            }
                        count_anaylze=0
                        if isinstance(group[group_id], list):
                            for text_post in group[group_id]:
                                # Анализируем посты группы
                                text_post = deEmojify(text_post)
                                text_post = mister_cleaner(text_post)
                                print(text_post)
                                print(len(text_post))
                                print("===")
                                if len(text_post) > 20:
                                    count_anaylze +=1
                                    auto = autoaression.AutoaggressionAnalysis(text_post)
                                    group_anaysis['spontaneous_aggression'].append(auto.spontaneous_aggression())
                                    group_anaysis['depression'].append(auto.depression())
                                    group_anaysis['balance'].append(auto.balance())
                                    group_anaysis['emotional_lability'].append(auto.emotional_lability())


                        group_result.append({group_id:
                                                {
                                    'spontaneous_aggression':np.average(group_anaysis['spontaneous_aggression']),
                                    'depression':np.average(group_anaysis['depression']),
                                    'balance':np.average(group_anaysis['balance']),
                                    'emotional_lability':np.average(group_anaysis['emotional_lability']),
                                                'count_post': count_anaylze
                                                }
                                                })
                        
                            
        return group_result


clearing = Pipeline([
    ('user_groups', AutoreggressionPipeline()),
])
tff = clearing.transform(tf)

увидеть пост от влада – значит, стать свидетелем чего-то поистине удивительного! : надо помочь человеку! по копеечке, но можем собрать неплохую сумму, давайте!   когда ещё такое будет? 
185
===
вай нот
7
===
хоп ла лайла!
13
===
эть!  классный воскресный своп, куда можно принести растения, книги, неиспользованные блокноты, пряжу, укулеле, бисер и т.д.  подробности ниже   при этом не обязательно приходить с вещами – можно и без них. главное иметь желание принять новую нужную вещь в свои руки 
267
===

0
===
короче, ребяты! беспрецедентный розыгрыш в ц!  можно выиграть:  • один билет на новогоднюю вечеринку на пушке • второй билет со скидкой 50% • 3 часа счастья, которыми можно будет сразу же воспользоваться все той же ночью • а б о н е м е н т на целый месяц с открытой датой!  победителей всего 4, скорее читайте условия и делайте репост, ну! 
341
===
питерские девчонки, эй!
23
===
знайте, что вы все очень милые, спасибо вам за это! 
52
===
зову на клёвый ништяк в циферблате!  устроили м

еще ближе к тебе!!!
19
===
анютка, ты у меня самая красивая!! я тебя люблю!
48
===
как же мне все надоело!!!!
26
===
как же мне все надоела!!!!
26
===
затащил анюта, все ради тебя, люблю очень сильно!!.
51
===
еще не много, еще чут-чуть!!!! надо тащить, ой как надо!
56
===
девочка моя я люблю тебя.
25
===
затащил.
8
===
надо затащить, иначе никак!
27
===
я люблю тебя.
13
===
было всё.
9
===
от сессии до сессии живут студенты весело.
42
===
рабкин.
7
===
опять матан.
12
===
любимая, с праздником тебя.
27
===
урааа, сдал анюта, люблю тебя очень сильно!
43
===
я самый счастливый человек на свете, ведь у меня есть ты.
57
===
я люблю тебя сильно, очень сильно ты моя жизнь!!!
49
===
здесь даже солнца не видно!!!
29
===
курсач-жопа!!!!!
16
===
я люблю тебя!
13
===
бегом по дворам...
18
===
я верю в себя!
14
===
за то я верю!!!
15
===
хочу к тебе!!!!
15
===
все меняется ничего не меняется
31
===
йоу, первое выступление за 2.5 года ты знаешь, что делать
57
===
сегодня мне 22 и это первый раз, к

===
с днем рождения!!!
18
===
от души я тебя поздравляю, веселись в этот день, не скучай! я желаю тебе столько счастья, чтоб оно пролилось через край. пусть хорошее все запомнится, все задуманное – пусть исполнится, пусть глаза твои счастьем светятся, и хорошие люди встретятся!
248
===
нажмите на анимашку, чтобы прочитать сообщение. : 
50
===
я сделал тебе татуировку! нажми, чтобы посмотреть и оценить! 
61
===
с днем рождения!  и с наступающим новым годом! удачного года в личной жизни и учебе! и всяких вкусняшек везде и не каких проблем:
129
===
с днюхой,миш!и заодно,раз тебе повезло втакой день родиться - с наступающим!удачи,радости и всего самого лучшего
112
===
с днем рождения желаю тебе ярких приключений  с наступающим новым годом!!
73
===
с днем рождения!
16
===
эта, пожалуй, моя любимая^^
27
===
приглашает стать ее соседом! 
29
===

0
===
спасибо!
8
===
спасибо! =^.^=
14
===
экстэрминатус наше спасение.
28
===
ересь!!!была.
13
===
"все вокруг" подразумевает и его
32
===
ты же так

ну варечка генийка конечно
26
===
как найти себе прекрасного мастеринг-инженера? а как найти себе прекрасного мастеринг-инженера, если ты бедный?  я доделал песню, которая самому мне очень нравится, хочу ее быстро выпустить синглом, но ещё хочу, чтобы она отлично звучала. прошлый альбом я, как смог, отмастерил сам, но честно сказать, у меня не только образования, у меня и колонок мониторных нет, так что хотелось бы сегодня без этого. если вы очень хороший и достаточно состоятельный мастеринг-инженер, напишите мне, пожалуйста, я пришлю вам трек, вдруг, договоримся.
536
===
идеальный фестиваль в сентябре
30
===
водитель убера рассказал, что его знакомый из органов сходил тоже помитинговать на той неделе. «это как это?!» — удивился мой водитель. «так за денежку же, почему бы нет», — разоткровенничался его знакомый.  а? не изящно ли?
224
===
как известно, сегодня день сони — праздник близкой нам финской культуры, широко отмечаемый прямо сейчас в центре москвы. зрелище удивительное, историч

торшеры! у вас есть торшеры? большие, маленькие. на пятницу. на концерт.  также интересны пластиковые бутылки цветов зеленого и коричневого.
140
===
как будто бы, важное время. про смерть уже все рассказано, и все правда. что, например?  что если ты придумал слова и музыку песням, все сам записал, свел и смастерил, договорился о фотосете с любимой фотографиней и сфоткался, выбрал обложку, договорился о разогреве у любимой группы, о съемке клипов, сделал себе плакат и графику, договорился с идеальной площадкой в городе о концерте с идеальной группой о поддержке этого концерта, и осталось только отослать все стриминговым сервисам, написать посты, обновить и отрепетировать лайв, не забросить работу в школе и понять, зачем ты вообще все это делаешь, то хочется рассказать обо всем маме, но уже не получится.  мама много знала про смерть, но больше про тоску. по сыну, по брату, по отцу. и не то что бы много про это рассказывала, но куда тут денешься. так что я все время угадываю в себе то, пр

...

===
не жизнь, а монтаж андрея банина
32
===
salut, cest encore moi!
23
===
достаточно было просто выйти из дома
36
===
я таю сам, как тает снег, я сам, как утро, брови хмурю.
55
===
искренность рождается в условиях последней крайности
52
===
и мы поставим свечи в изголовье погибших от невиданной любви
60
===
🥀 narcissism syndrome
21
===
выглядите, как будто из европы, но вы из тушино
47
===
сначала выглядишь импозантно в весеннем пальто, а потом - глупо с температурой 39
81
===
у нас 6 рублей и великий потенциал
34
===
человек хотел написать про любовь, а написал про пиво
53
===
блин что язв странные признания в любви
39
===
стресс - это я !
16
===
поздравляю нас, космонавтов
27
===
спасибо, за прекрасные фотографии. обработка моя, я первый раз в руки взял фотошоп, сори :d
91
===
привееееет ,я тоже
18
===
на случай важных переговоров
28
===
все правильно говорят0
22
===
класс :d
8
===
аааааа
6
===
вот да
6
===
вот да о.о
10
===
я влюбился.. гениальная композиция
34
===
миp, в котоpом м

это стоит почитать только после просмотра, я предупреждаю
57
===
это здорово
11
===
тебе понравится.  или нет.  
28
===
это прекрасно и для меня
24
===
это важно и просто
18
===
это мне нравится психология
27
===
тебе понравится.  или нет.  
28
===
это ностальгия
14
===
это инструкция
14
===
тебе понравится. или нет.  
27
===
это о спокойствии и любви
25
===
тебе понравится.  или нет.  
28
===
в этом году открывает октябрь лучшая тыковка всея скрэнтона
59
===
это мне нравится
16
===
это мне не стыдно!!!
20
===
это я всё ещё люблю даниила хармса
34
===
тебе понравится. или нет. 
26
===
тебе понравится.  или нет.  
28
===
тебе понравится.  или нет.  
28
===
видеофайл этого месяца!
23
===
ждала с нетерпением октябрь, чтобы закинуть этот видос сюда, смотреть и смеяться.  тыквенный месяц, ура!
104
===
это детство
11
===
это озеро и горы 
17
===
тебе понравится.  или нет.  
28
===
а если перейти на сайт youtube, то можно с субтитрами глянуть
61
===
это любимое
11
===
тебе понравится.  или не

кто мне помогает в моем деле?  привет, призывниксегодня я тебе расскажу про мою команду. вместе мы помогаем десяткам ребят решать вопрос с армией законно.  хочу тебя огорчить, для получения военного билета недостаточно просто прийти на медкомиссию и пожаловаться на недомогание. нужно обязательно готовить правильные медицинские и юридические документы, проходить обследования от военкомата, подтверждать диагноз, а потом еще и проходить призывную комиссию. нередко, когда приходится обжаловать решения военкомата или нарушения с их стороны.  взаимодействие с военкоматом с целью получения военника без службы часто требует от призывника большого опыта в этой сфере и знания российского законодательства. каждый шаг нужно проходить правильно, чтобы в итоге получить свой законный военник.  в этом нелёгком деле мне помогает моя надежная команда, которая работает, как слаженный механизм.  за годы моей практики в работу включились:  два замечательных врача - анастасия и виктория. кстати обе закончил

когда заработают военкоматы и начнут забирать в армию?  кроме того, в этом виде рассказал: - почему сроки призыва дальше сдвигать уже не будут? - у каких призывников могут быть проблемы в этом призыве? - и что делать, чтобы не попасть под гребенку военкомата?  если посмотрев это видео, ты понял, что хочешь обезопасить себя и получить военник, пиши мне в личные сообщения   могу провести тебе консультацию бесплатно или ответить на твои вопросы. используй возможность пока еще не поздно.  султан мирзаев. на связи.
515
===
сегодня выбрался на природу. нужно почаще это делать. поднимает настроение и в целом сейчас это очень полезно.   
113
===
военкоматы россии перевели на удаленную работу  по информации иа "тасс", министерство обороны рф объявило о переводе военкоматов на дистанционную работу. официально призыв в армию из-за коронавируса не отменяется, но начало медицинских и призывных комиссий из-за короновируса отложено. пока сотрудники комиссариатов будут из дома работать с личными делам

думаешь заняться военником, когда закончится отсрочка?  у меня для тебя плохие новости - скорее всего не получится.  посмотри 3-х минутное видео и поймешь почему.   если все-таки решишь не откладывать, пиши мне в лс, подскажу, как лучше действовать.   ps: если понравилось видео, ставь лайк, пиши комментарий, делай репост, буду записывать еще.   султан мирзаев.  до связи! 🤙
375
===
5 шагов по получению военника и приглашение на закрытую онлайн-трансляцию   в этом видео я кратко рассказываю о 5 основных шагах для получения военного билета! наверное сам понимаешь, что за 4 минуты весь процесс не расскажешь. поэтому  сегодня в 19:30 по мск я поведу закрытую онлайн-трансляцию на тему: как получить военный билет законно и без службы в армии".   для того, чтобы посмотреть вебинар, ты должен быть подписан на группу . это важно, т.к. трансляция будет закрытой, чтобы присутствовали только свои.🤜🤛  на трансляции мы разберем: - какие изменения законодательства будут в этом году и чем тебе это гроз

In [96]:
tff

[{741967: {'spontaneous_aggression': 0.08433093908215113,
   'depression': 0.5420544172163939,
   'balance': 10.975092628473101,
   'emotional_lability': 0.8638074436809329,
   'count_post': 50}},
 {1189718: {'spontaneous_aggression': -0.13798762626262842,
   'depression': -0.23553282828282698,
   'balance': 12.291321338383838,
   'emotional_lability': -0.4634468434343442,
   'count_post': 6}},
 {1308839: {'spontaneous_aggression': -0.0661202752358303,
   'depression': -0.06216246568236848,
   'balance': 14.409581060943571,
   'emotional_lability': 0.1746092574094373,
   'count_post': 41}},
 {1576851: {'spontaneous_aggression': nan,
   'depression': nan,
   'balance': nan,
   'emotional_lability': nan,
   'count_post': 0}},
 {5526137: {'spontaneous_aggression': 1.1511197564780882,
   'depression': 0.398848929635712,
   'balance': 11.408048625347622,
   'emotional_lability': 0.19335259526020904,
   'count_post': 29}},
 {7043561: {'spontaneous_aggression': 1.5561620318408789,
   'depress